# Multimodal Embedding Models

In [1]:
import io
import base64
import boto3
import json
import os
from PIL import Image
from typing import List

In [2]:
session = boto3.Session()
client = session.client("bedrock-runtime")

In [9]:
def readFileAsBase64(file_path):
    with Image.open(file_path) as img:
        img = img.convert("RGB")  # Convert to RGB mode
        resized_img = img.resize((200,200))
        buffered = io.BytesIO()
        resized_img.save(buffered, format="JPEG")
        base64_string = base64.b64encode(buffered.getvalue()).decode("utf8")
    return base64_string

def construct_bedrock_body(base64_string: str) -> str:
    """Construct the request body for the Bedrock API."""
    return json.dumps(
        {
            "inputImage": base64_string,
            "embeddingConfig": {"outputEmbeddingLength": 1024},
        }
    )

def get_image_embedding(file_path: str) -> List[float]:
    """Get image embedding from Bedrock API."""
    base64_string = readFileAsBase64(file_path)
    body = construct_bedrock_body(base64_string)

    try:
        response = client.invoke_model(
            body=body,
            modelId="amazon.titan-embed-image-v1",
            accept="application/json",
            contentType="application/json",
        )

        response_body = json.loads(response.get("body").read())
        return response_body["embedding"]
    except Exception as e:
        raise ValueError(f"Error raised by image embedding endpoint: {e}")

In [13]:
embedding = get_image_embedding("../../images/mteb_leaderboard.png")

In [15]:
print(len(embedding))

1024
